# Configuration & Tools

In [ ]:
import sys
IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    !apt install python-opengl
    !apt install ffmpeg
    !apt install xvfb
    !pip install pyvirtualdisplay
    from pyvirtualdisplay import Display
    
    # Start virtual display
    dis = Display(visible=0, size=(600, 400))
    dis.start()

## Import modules

In [ ]:
import random
from collections import deque
from typing import Deque, Dict, List, Tuple

import gym
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from IPython.display import clear_output
from torch.distributions import Normal

## Set random seed

In [ ]:
if torch.backends.cudnn.enabled:
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

seed = 777
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

## Replay buffer
Typically, people implement replay buffers with one of the following three data structures:

- collections.deque
- list
- numpy.ndarray

**deque** is very easy to handle once you initialize its maximum length (e.g. deque(maxlen=buffer_size)). However, the indexing operation of deque gets terribly slow as it grows up because it is [internally doubly linked list](https://wiki.python.org/moin/TimeComplexity#collections.deque). On the other hands, **list** is an array, so it is relatively faster than deque when you sample batches at every step. Its amortized cost of Get item is [O(1)](https://wiki.python.org/moin/TimeComplexity#list).

Last but not least, let's see **numpy.ndarray**. numpy.ndarray is even faster than list due to the fact that it is [a homogeneous array of fixed-size items](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html#numpy.ndarray), so you can get the benefits of [locality of reference](https://en.wikipedia.org/wiki/Locality_of_reference), . Whereas list is an array of pointers to objects, even when all of them are of the same type.

Here, we are going to implement a replay buffer using numpy.ndarray.

Reference: 
- [OpenAI spinning-up](https://github.com/openai/spinningup/blob/master/spinup/algos/sac/sac.py#L10)
- [rainbow-is-all-you-need](https://render.githubusercontent.com/view/ipynb?commit=032d11277cf2436853478a69ca5a4aba03202598&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f437572742d5061726b2f7261696e626f772d69732d616c6c2d796f752d6e6565642f303332643131323737636632343336383533343738613639636135613461626130333230323539382f30312e64716e2e6970796e62&nwo=Curt-Park%2Frainbow-is-all-you-need&path=01.dqn.ipynb&repository_id=191133946&repository_type=Repository#Replay-buffer)

In [ ]:
class ReplayBuffer:
    """A simple numpy replay buffer."""

    def __init__(self, obs_dim: int, size: int, batch_size: int = 32):
        """Initialize."""
        # ==================================== Your Code (Begin) ====================================
        self.obs_buf = np.zeros([size, obs_dim], dtype=np.float32)
        self.next_obs_buf = np.zeros([size, obs_dim], dtype=np.float32)
        self.acts_buf = np.zeros([size], dtype=np.float32)
        self.rews_buf = np.zeros([size], dtype=np.float32)
        self.done_buf = np.zeros([size], dtype=np.float32)
        self.max_size, self.batch_size = size, batch_size
        self.ptr, self.size = 0, 0
        # ==================================== Your Code (End) ====================================


    def store(self,
        obs: np.ndarray,
        act: np.ndarray, 
        rew: float, 
        next_obs: np.ndarray, 
        done: bool,
    ):
        """Store the transition in buffer."""
        # ==================================== Your Code (Begin) ====================================
        self.obs_buf[self.ptr] = obs
        self.next_obs_buf[self.ptr] = next_obs
        self.acts_buf[self.ptr] = act
        self.rews_buf[self.ptr] = rew
        self.done_buf[self.ptr] = done
        self.ptr = (self.ptr + 1) % self.max_size
        self.size = min(self.size + 1, self.max_size)
        # ==================================== Your Code (End) ====================================


    def sample_batch(self) -> Dict[str, np.ndarray]:
        """Randomly sample a batch of experiences from memory."""
        # ==================================== Your Code (Begin) ====================================
        idxs = np.random.choice(self.size, size=self.batch_size, replace=False)
        return dict(
            obs=self.obs_buf[idxs],
            next_obs=self.next_obs_buf[idxs],
            acts=self.acts_buf[idxs],
            rews=self.rews_buf[idxs],
            done=self.done_buf[idxs]
        )
        # ==================================== Your Code (End) ====================================


    def __len__(self) -> int:
        # ==================================== Your Code (Begin) ====================================
        return self.size
        # ==================================== Your Code (End) ====================================


### Explanation: Replay Buffer Implementation

The **ReplayBuffer** is a critical component for off-policy algorithms like DDPG. It stores past experiences (state, action, reward, next_state, done) and allows the agent to learn from random mini-batches of experiences.

**Key Features:**
- Uses `numpy.ndarray` for efficient memory access (O(1) indexing)
- Circular buffer implementation: when full, oldest experiences are overwritten
- Random sampling breaks temporal correlations in sequential data
- Stores transitions in separate arrays for each component

**Why Replay Buffer?**
- **Stability**: Random sampling decorrelates sequential experiences
- **Efficiency**: Reuses past experiences multiple times
- **Memory**: Fixed size prevents unbounded memory growth

## OU Noise
**Ornstein-Uhlenbeck** process generates temporally correlated exploration, and it effectively copes with physical control problems of inertia.

$$
dx_t = \theta(\mu - x_t) dt + \sigma dW_t
$$

Reference: 
- [Udacity github](https://github.com/udacity/deep-reinforcement-learning/blob/master/ddpg-pendulum/ddpg_agent.py)
- [Wiki](https://en.wikipedia.org/wiki/Ornstein%E2%80%93Uhlenbeck_process)

In [ ]:
import copy
class OUNoise:
    """Ornstein-Uhlenbeck process.
    Taken from Udacity deep-reinforcement-learning github repository:
    https://github.com/udacity/deep-reinforcement-learning/blob/master/
    ddpg-pendulum/ddpg_agent.py
    """

    def __init__(
        self, 
        size: int, 
        mu: float = 0.0, 
        theta: float = 0.15, 
        sigma: float = 0.2,
    ):
        """Initialize parameters and noise process."""
        self.state = np.float64(0.0)
        self.mu = mu * np.ones(size)
        self.theta = theta
        self.sigma = sigma
        self.reset()

    def reset(self):
        """Reset the internal state (= noise) to mean (mu)."""
        self.state = copy.copy(self.mu)

    def sample(self) -> np.ndarray:
        """Update internal state and return it as a noise sample."""
        x = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.array(
            [random.random() for _ in range(len(x))]
        )
        self.state = x + dx
        return self.state

### Explanation: Ornstein-Uhlenbeck Noise

The **OU Noise** generates temporally correlated exploration noise, which is more effective than uncorrelated Gaussian noise for physical control tasks with inertia.

**Formula:** $dx_t = \theta(\mu - x_t) dt + \sigma dW_t$

**Parameters:**
- $\mu$: mean value (usually 0)
- $\theta$: rate of mean reversion (how quickly noise returns to mean)
- $\sigma$: volatility/scale of random fluctuations

**Why OU Noise for DDPG?**
- Creates smooth, correlated exploration
- Better for continuous control with momentum/inertia
- Prevents abrupt action changes that could destabilize physical systems

# PPO

- PPO: [J. Schulman et al., "Proximal Policy Optimization Algorithms." arXiv preprint arXiv:1707.06347, 2017.](https://arxiv.org/abs/1707.06347.pdf)
- TRPO: [Schulman, John, et al. "Trust region policy optimization." International conference on machine learning. 2015.](http://proceedings.mlr.press/v37/schulman15.pdf)

There are two kinds of algorithms of PPO: PPO-Penalty and PPO-Clip. Here, we'll implement PPO-clip version.

TRPO computes the gradients with a complex second-order method. On the other hand, PPO tries to solve the problem with a first-order methods that keep new policies close to old. To simplify the surrogate objective, let $r(\theta)$ denote the probability ratio

$$ L^{CPI}(\theta) = \hat {\mathbb{E}}_t \left [ {\pi_\theta(a_t|s_t) \over \pi_{\theta_{old}}(a_t|s_t)} \hat A_t\right] = \hat {\mathbb{E}}_t \left [ r_t(\theta) \hat A_t \right ].$$

The objective is penalized further away from $r_t(\theta)$

$$ L^{CLIP}(\theta)=\hat {\mathbb{E}}_t \left [ \min(r_t(\theta) \hat A_t, \text{clip}(r_t(\theta), 1-\epsilon, 1+\epsilon)\hat A_t) \right ] $$

If the advantage is positive, the objective will increase. As a result, the action becomes more likely. If advantage is negative, the objective will decrease. AS a result, the action becomes less likely.

### Explanation: PPO Network Architecture

**PPO Actor Network:**
- Outputs a **stochastic policy**: mean ($\mu$) and standard deviation ($\sigma$) of a Gaussian distribution
- Uses `tanh` on mean to bound actions to [-1, 1]
- `log_std` is clamped to prevent numerical instability
- Samples actions from $\mathcal{N}(\mu, \sigma^2)$ for exploration

**Critic Network:**
- Estimates the state value function $V(s)$
- Used to compute advantages: $A(s,a) = Q(s,a) - V(s)$
- Simpler than Q-networks (doesn't need action as input)

**Key Design Choices:**
- Separate networks for actor and critic (more stable than shared)
- Uniform initialization on final layers (prevents large initial gradients)
- ReLU activations in hidden layers (standard for deep RL)

# DDPG 

[T. P. Lillicrap et al., "Continuous control with deep reinforcement learning." arXiv preprint arXiv:1509.02971, 2015.](https://arxiv.org/pdf/1509.02971.pdf)

Deep Q Network(DQN)([Mnih et al., 2013;2015](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf)) algorithm is combined advances in deep learning with reinforcement learning. However, while DQN solves problems with high-dimentional observation spaces, it can only handle discrete and low-dimentional action spaces because of using greedy policy. For learning in high-dimentional and continous action spaces, the authors combine the actor-critic approach with insights from the recent success of DQN. Deep DPG(DDPG) is based on the deterministic policy gradient(DPG) algorithm ([Silver et al., 2014](http://proceedings.mlr.press/v32/silver14.pdf)). 

### Deterministic policy gradient
The DPG algorithm maintains a parameterized actor function $\mu(s|\theta^{\mu})$ which specifies the current policy by deterministically mapping states to a specific action. The critic $Q(s, a)$ is learned using the Bellman equation as in Q-learning. The actor is updated by following the applying the chain rule to the expected return from the start distribution $J$ with respect to the actor parameters

$$
\begin{align*}
\nabla_{\theta^{\mu}}J &\approx E_{s_t\sim\rho^\beta} [\nabla_{\theta^{\mu}} Q(s,a|\theta^Q)|_{s=s_t, a=\mu(s_t|\theta^\mu)}] \\
&= E_{s_t\sim\rho^\beta} [\nabla_{a} Q(s,a|\theta^Q)|_{s=s_t, a=\mu(s_t)} \nabla_{\theta^{\mu}} \mu(s|\theta^\mu)|_{s=s_t}]
\end{align*}
$$

### Soft update target network
Since the network $(Q(s,a|\theta^Q)$ being updated is also used in calculating the target value, the Q update is prone to divergence. To avoid this, the authors use **the target network** like DQN, but modified for actor-critic and using **soft target updates**. Target netwokrs is created by copying the actor and critic networks, $Q'(s,a|\theta^{Q'})$ and $\mu'(s|\theta^{\mu`})$ respectively, that are used for calculating the target values. The weights of these target networks are then updated by having them slowly track the learned networks:

$$
\theta' \leftarrow \tau \theta + (1 - \tau)\theta' \ \ \ {with} \ \tau \ll 1.
$$

It greatly improves the stability of learning.

### Exploration for continuous action space
An advantage of offpolicies algorithms such as DDPG is that we can treat the problem of exploration independently from the learning algorithm. The authors construct an exploration policy $\mu'$ by adding noise sampled from a noise process $\mathcal{N}$ to the actor policy

$$
\mu'(s_t) = \mu(s_t|\theta^{\mu}_t) + \mathcal{N}
$$

$\mathcal{N}$ can be chosen to suit the environment. The authors used **Ornstein-Uhlenbeck process** to generate temporally correlated exploration.

### Explanation: DDPG Network Architecture

**DDPG Actor Network:**
- Outputs a **deterministic policy**: $\mu(s)$
- Uses `tanh` activation to bound actions to [-1, 1]
- No stochasticity in the policy (exploration via noise)

**DDPG Critic Network:**
- Estimates the Q-function: $Q(s, a)$
- Takes **both state and action** as input (concatenated)
- Used to guide actor training via policy gradient

**DDPG vs PPO Networks:**
| Aspect | PPO | DDPG |
|--------|-----|------|
| Actor Output | Stochastic (μ, σ) | Deterministic (μ) |
| Critic Input | State only | State + Action |
| Critic Output | V(s) | Q(s,a) |
| Exploration | Policy entropy | External noise |

**Target Networks:**
DDPG uses target networks (not shown in architecture, but used in training) to stabilize learning.

# Networks


### Explanation: PPO Mini-Batch Iterator

The `ppo_iter` function yields mini-batches from collected rollout data for multiple epochs.

**Why Multiple Epochs?**
- PPO is **on-policy** but reuses data multiple times
- Clipping prevents too-large policy updates
- More sample efficient than vanilla policy gradients

**Process:**
1. Collect `rollout_len` steps of experience
2. For each `epoch`:
   - Split data into `mini_batch_size` chunks
   - Update policy on each mini-batch
3. Discard old data and collect new rollouts

**Trade-off:**
- More epochs = more updates per data
- But too many epochs = policy drift (mitigated by clipping)

## PPO Networks

### Explanation: PPO Agent Implementation

**Key Components:**

**1. Generalized Advantage Estimation (GAE):**
```python
delta = reward + gamma * V(next_state) - V(state)
gae = delta + gamma * tau * gae
```
- $\tau$ controls bias-variance trade-off
- Higher $\tau$ = more variance, less bias
- Returns = advantages + values

**2. Clipped Surrogate Objective:**
```python
ratio = exp(new_log_prob - old_log_prob)
L = min(ratio * advantage, clip(ratio, 1-ε, 1+ε) * advantage)
```
- Prevents too-large policy updates
- $\epsilon$ typically 0.1-0.3

**3. Entropy Regularization:**
- Encourages exploration
- Loss: `actor_loss - entropy_weight * entropy`
- Higher entropy = more random actions

**Training Loop:**
1. Collect `rollout_len` transitions
2. Compute returns and advantages with GAE
3. Normalize advantages (improves stability)
4. Update policy for `epoch` iterations
5. Clear memory and repeat

**PPO Advantages:**
- ✅ Stable and robust
- ✅ Few hyperparameters
- ✅ Works well across many tasks
- ❌ On-policy (less sample efficient than DDPG)

We will use two separated networks for actor and critic respectively. The actor network consists of two fully connected hidden layer with ReLU branched out two fully connected output layers for mean and standard deviation of Gaussian distribution. Pendulum-v0 has only one action which has a range from -2 to 2. In order to fit the range, the actor outputs the mean value with tanh. The result will be scaled in ActionNormalizer class. On the one hand, the critic network has three fully connected layers as two hidden layers (ReLU) and an output layer. One thing to note is that we initialize the last layers' weights and biases as uniformly distributed.

### Explanation: DDPG Agent Implementation

**Key Components:**

**1. Deterministic Policy Gradient:**
```python
actor_loss = -Q(s, actor(s)).mean()
```
- Maximizes Q-value by adjusting actor parameters
- Chain rule: $\nabla_\theta J = E[\nabla_a Q(s,a) \nabla_\theta \mu(s)]$

**2. Target Networks:**
- Separate target networks for actor and critic
- Soft updates: $\theta' \leftarrow \tau \theta + (1-\tau) \theta'$
- Stabilizes training (like DQN)

**3. Replay Buffer:**
- Stores all past experiences
- Samples random mini-batches
- Breaks temporal correlations

**4. Exploration:**
- Initial random steps: `initial_random_steps`
- OU noise added to actions during training
- No noise during testing

**Training Loop:**
1. Select action: $a = \mu(s) + \mathcal{N}$ (with noise)
2. Store transition in replay buffer
3. Sample mini-batch from buffer
4. Compute target: $y = r + \gamma Q'(s', \mu'(s'))$
5. Update critic: minimize $(Q(s,a) - y)^2$
6. Update actor: maximize $Q(s, \mu(s))$
7. Soft update target networks

**DDPG Advantages:**
- ✅ Off-policy (sample efficient)
- ✅ Deterministic policy (faster convergence)
- ✅ Works well for continuous control
- ❌ Sensitive to hyperparameters
- ❌ Can be unstable (needs careful tuning)

In [ ]:
def init_layer_uniform(layer: nn.Linear, init_w: float = 3e-3) -> nn.Linear:
    """Init uniform parameters on the single layer."""
    # ==================================== Your Code (Begin) ====================================
        low = self.action_space.low
        high = self.action_space.high
        
        action = low + (action + 1.0) * 0.5 * (high - low)
        action = np.clip(action, low, high)
        
        return action
    # ==================================== Your Code (End) ====================================


class PPOActor(nn.Module):
    def __init__(
        self, 
        in_dim: int, 
        out_dim: int, 
        log_std_min: int = -20,
        log_std_max: int = 0,
    ):
        """Initialize."""
        super(PPOActor, self).__init__()

        # ==================================== Your Code (Begin) ====================================
        low = self.action_space.low
        high = self.action_space.high
        
        action = 2 * (action - low) / (high - low) - 1
        action = np.clip(action, -1.0, 1.0)
        
        return action
        # ==================================== Your Code (End) ====================================


    def forward(self, state: torch.Tensor) -> torch.Tensor:
        """Forward method implementation."""
        
        # ==================================== Your Code (Begin) ====================================
        # ==================================== Your Code (End) ====================================



class Critic(nn.Module):
    def __init__(self, in_dim: int):
        """Initialize."""
        super(Critic, self).__init__()
        # ==================================== Your Code (Begin) ====================================
        # ==================================== Your Code (End) ====================================


    def forward(self, state: torch.Tensor) -> torch.Tensor:
        """Forward method implementation."""
        # ==================================== Your Code (Begin) ====================================
        # ==================================== Your Code (End) ====================================


### Explanation: Action Normalizer

The `ActionNormalizer` wrapper transforms actions between the network's output range [-1, 1] and the environment's actual action range [low, high].

**Why Normalize Actions?**
- Neural networks with `tanh` output naturally produce values in [-1, 1]
- Environments have arbitrary action ranges (e.g., Pendulum: [-2, 2])
- Normalization allows the network to work with a standard range

**Forward Transformation** (network → environment):
```python
action = low + (action + 1) * 0.5 * (high - low)
```
Maps [-1, 1] → [low, high]

**Reverse Transformation** (environment → network):
```python
action = 2 * (action - low) / (high - low) - 1
```
Maps [low, high] → [-1, 1]

**Example:** Pendulum-v1 actions are in [-2, 2]
- Network outputs 0.5
- Normalized: -2 + (0.5 + 1) * 0.5 * 4 = 1.0
- Environment receives action = 1.0 ✅

## DDPG Networks
We are going to use two separated networks for actor and critic. The actor network has three fully connected layers and three non-linearity functions, **ReLU** for hidden layers and **tanh** for the output layer. On the other hand, the critic network has three fully connected layers, but it used two activation functions for hidden layers **ReLU**. Plus, its input sizes of critic network are sum of state sizes and action sizes. One thing to note is that we initialize the final layer's weights and biases so that they are **uniformly distributed.**

In [ ]:
class DDPGActor(nn.Module):
    def __init__(
        self, 
        in_dim: int, 
        out_dim: int,
        init_w: float = 3e-3,
    ):
        """Initialize."""
        super(DDPGActor, self).__init__()
        
        # ==================================== Your Code (Begin) ====================================
        # 1. set the hidden layers
        # 2. init hidden layers uniformly       
        # ==================================== Your Code (End) ====================================

    def forward(self, state: torch.Tensor) -> torch.Tensor:
        """Forward method implementation."""
        # ==================================== Your Code (Begin) ====================================
        # use a tanh function as a ativation function for output layer 
        # ==================================== Your Code (End) ====================================
    
    
class DDPGCritic(nn.Module):
    def __init__(
        self, 
        in_dim: int, 
        init_w: float = 3e-3,
    ):
        """Initialize."""
        super(DDPGCritic, self).__init__()
        
        # ==================================== Your Code (Begin) ====================================
        # 1. set the hidden layers
        # 2. init hidden layers uniformly       
        # ==================================== Your Code (End) ====================================

    def forward(
        self, state: torch.Tensor, action: torch.Tensor
    ) -> torch.Tensor:
        """Forward method implementation."""
        # ==================================== Your Code (Begin) ====================================
        # notice that this value function is Q(s, a)
        # ==================================== Your Code (End) ====================================


# Agents 

## PPO Agent
Here is a summary of PPOAgent class.

| Method           | Note                                                 |
|---               |---                                                   |
|select_action     | select an action from the input state.               |
|step              | take an action and return the response of the env.   |
|update_model      | update the model by gradient descent.                |
|train             | train the agent during num_frames.                   |
|test              | test the agent (1 episode).                          |
|_plot             | plot the training progresses.                        |


PPO updates the model several times(`epoch`) using the stacked memory. By `ppo_iter` function, It yield the samples of stacked memory by interacting a environment.

In [ ]:
def ppo_iter(
    epoch: int,
    mini_batch_size: int,
    states: torch.Tensor,
    actions: torch.Tensor,
    values: torch.Tensor,
    log_probs: torch.Tensor,
    returns: torch.Tensor,
    advantages: torch.Tensor,
):
    """Yield mini-batches."""
    
    # ==================================== Your Code (Begin) ====================================
        
    # ==================================== Your Code (End) ====================================


In [ ]:
class PPOAgent:
    """PPO Agent.
    Attributes:
        env (gym.Env): Gym env for training
        gamma (float): discount factor
        tau (float): lambda of generalized advantage estimation (GAE)
        batch_size (int): batch size for sampling
        epsilon (float): amount of clipping surrogate objective
        epoch (int): the number of update
        rollout_len (int): the number of rollout
        entropy_weight (float): rate of weighting entropy into the loss function
        actor (nn.Module): target actor model to select actions
        critic (nn.Module): critic model to predict state values
        transition (list): temporory storage for the recent transition
        device (torch.device): cpu / gpu
        total_step (int): total step numbers
        is_test (bool): flag to show the current mode (train / test)        
    """

    def __init__(
        self,
        env: gym.Env,
        batch_size: int,
        gamma: float,
        tau: float,
        epsilon: float,
        epoch: int,
        rollout_len: int,
        entropy_weight: float,
    ):
        """Initialize."""
        # ==================================== Your Code (Begin) ====================================
        # 1. set hyperparameters
        # 2. check device: cpu/GPU
        # 3. init actor critic networks
        # 4. set Optimizer for each network
        # 5. consider memory for training
        # 6. set total step counts equal to 1
        # 7. define a mode for train/test
        # ==================================== Your Code (End) ====================================

    
        

    def select_action(self, state: np.ndarray) -> np.ndarray:
        """Select an action from the input state."""
        # ==================================== Your Code (Begin) ====================================
        # 1. select action for train or test mode
        # 2. if you are in train mode take care of filing considered memory
        # ==================================== Your Code (End) ====================================


    def step(self, action: np.ndarray) -> Tuple[np.ndarray, np.float64, bool]:
        """Take an action and return the response of the env."""
        # ==================================== Your Code (Begin) ====================================
        
        # ==================================== Your Code (End) ====================================


    def update_model(
        self, next_state: np.ndarray
    ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """Update the model by gradient descent."""
        # ==================================== Your Code (Begin) ====================================
        # 1. set device
        # 2. for each step:
        # 3.    calculate ratios
        # 4.    calculate actor_loss
        # 5.    calculate entropy
        # 6.    calculate critic_loss
        # 7.    Train  critic
        # 8.    Train actor
        # ==================================== Your Code (End) ====================================

    

    def train(self, num_frames: int, plotting_interval: int = 200):
        """Train the agent."""
        # ==================================== Your Code (Begin) ====================================
        # 1. set the status of trainig
        # 2. Reset environment
        # 3. for number of frames:
        # 4.    select an action
        # 5.    step in environment
        # 6.    update model
        # 7. terminate environment after training is finished
        # ==================================== Your Code (End) ====================================

        

    def test(self):
        """Test the agent."""
        # ==================================== Your Code (Begin) ====================================
        # 1. set the status of trainig
        # 2. Reset environment
        # 3. roll out one episode living in the environment and save frames for getting render
        # ==================================== Your Code (End) ====================================


    def _plot(
        self,
        frame_idx: int,
        scores: List[float],
        actor_losses: List[float],
        critic_losses: List[float],
    ):
        """Plot the training progresses."""
        # ==================================== Your Code (Begin) ====================================
        # 1. define a function for sub plots
        # 2. for each variable plot the specific subplot
        # 3. plot the whole figure
        # ==================================== Your Code (End) ====================================


## DDPG Agent
Here is a summary of DDPGAgent class.

| Method           | Note                                                 |
|---               |---                                                  |
|select_action     | select an action from the input state.               |
|step              | take an action and return the response of the env.   |
|update_model      | update the model by gradient descent.                |
|train             | train the agent during num_frames.                   |
|test              | test the agent (1 episode).                          |
|\_target_soft_update| soft update from the local model to the target model.|
|\_plot              | plot the training progresses.                        |

In [ ]:
class DDPGAgent:
    """DDPGAgent interacting with environment.
    
    Attribute:
        env (gym.Env): openAI Gym environment
        actor (nn.Module): target actor model to select actions
        actor_target (nn.Module): actor model to predict next actions
        actor_optimizer (Optimizer): optimizer for training actor
        critic (nn.Module): critic model to predict state values
        critic_target (nn.Module): target critic model to predict state values
        critic_optimizer (Optimizer): optimizer for training critic
        memory (ReplayBuffer): replay memory to store transitions
        batch_size (int): batch size for sampling
        gamma (float): discount factor
        tau (float): parameter for soft target update
        initial_random_steps (int): initial random action steps
        noise (OUNoise): noise generator for exploration
        device (torch.device): cpu / gpu
        transition (list): temporory storage for the recent transition
        total_step (int): total step numbers
        is_test (bool): flag to show the current mode (train / test)
    """
    def __init__(
        self,
        env: gym.Env,
        memory_size: int,
        batch_size: int,
        ou_noise_theta: float,
        ou_noise_sigma: float,
        gamma: float = 0.99,
        tau: float = 5e-3,
        initial_random_steps: int = 1e4,
    ):
        """Initialize."""

        # ==================================== Your Code (Begin) ====================================
        # 1. initialize hyper parameters, reply buffer and environment
        # 2. set device
        # 3. set target entropy, log alpha and alpha optimizer
        # 4. init actor network
        # 5. init value fuction (value critic)
        # 6. init OUNoise 
        # 7. set Optimizers
        # consider stroring transitions in memeory, counting steps and specify train/test mode
        # ==================================== Your Code (End) ====================================
    
    def select_action(self, state: np.ndarray) -> np.ndarray:
        """Select an action from the input state."""
        
        # ==================================== Your Code (Begin) ====================================
        # 1. check if initial random action should be conducted
        # 2. add noise for exploration during training
        # 3. store transition
        # return selected action
        # ==================================== Your Code (End) ====================================
    
    def step(self, action: np.ndarray) -> Tuple[np.ndarray, np.float64, bool]:
        """Take an action and return the response of the env."""
        # ==================================== Your Code (Begin) ====================================
        # step in environment and save transition in memory if you are not in test mode
        # ==================================== Your Code (End) ====================================
    
    def update_model(self) -> torch.Tensor:
        """Update the model by gradient descent."""
        # ==================================== Your Code (Begin) ====================================
        # 1. set device
        # 2. get a batch from memory and calculate the return
        # 3. calculate the loss for actor and critic networks
        # 4. update target
        # ==================================== Your Code (End) ====================================
    
    def train(self, num_frames: int, plotting_interval: int = 200):
        """Train the agent."""
        # ==================================== Your Code (Begin) ====================================
        # 1. set the status of trainig
        # 2. Reset environment
        # 3. for number of frames:
        # 4.    select an action
        # 5.    step in environment
        # 6.    update model
        # 7. plot the computed variables
        # 8. terminate environment after training is finished
        # ==================================== Your Code (End) ====================================
        
    def test(self):
        """Test the agent."""
        # ==================================== Your Code (Begin) ====================================
        # 1. set the status of trainig
        # 2. Reset environment
        # 3. roll out one episode living in the environment and save frames for getting render
        # ==================================== Your Code (End) ====================================
    
    def _target_soft_update(self):
        """Soft-update: target = tau*local + (1-tau)*target."""
        # ==================================== Your Code (Begin) ====================================
        
        # ==================================== Your Code (End) ====================================
    
    def _plot(
        self, 
        frame_idx: int, 
        scores: List[float], 
        actor_losses: List[float], 
        critic_losses: List[float], 
    ):
        """Plot the training progresses."""
        # ==================================== Your Code (Begin) ====================================
         
        # ==================================== Your Code (End) ====================================

# Environment
*ActionNormalizer* is an action wrapper class to normalize the action values ranged in (-1. 1). Thanks to this class, we can make the agent simply select action values within the zero centered range (-1, 1).

In [ ]:
class ActionNormalizer(gym.ActionWrapper):
    """Rescale and relocate the actions."""

    def action(self, action: np.ndarray) -> np.ndarray:
        """Change the range (-1, 1) to (low, high)."""
        # ==================================== Your Code (Begin) ====================================
        
        # ==================================== Your Code (End) ====================================

    

    def reverse_action(self, action: np.ndarray) -> np.ndarray:
        """Change the range (low, high) to (-1, 1)."""
        # ==================================== Your Code (Begin) ====================================
        
        # ==================================== Your Code (End) ====================================


You can see [the code](https://github.com/openai/gym/blob/master/gym/envs/classic_control/pendulum.py) and [configurations](https://github.com/openai/gym/blob/cedecb35e3428985fd4efad738befeb75b9077f1/gym/envs/__init__.py#L81) of Pendulum-v1 from OpenAI's repository.

In [ ]:
# environment
env_id = "Pendulum-v1"
env = gym.make(env_id)
env = ActionNormalizer(env)
env.seed(seed)

## PPO vs DDPG: Comprehensive Comparison

### Algorithm Type
| Algorithm | Type | Learning | Sample Efficiency |
|-----------|------|----------|-------------------|
| PPO | Policy Gradient | On-policy | Lower (needs fresh data) |
| DDPG | Actor-Critic | Off-policy | Higher (reuses old data) |

### Policy Type
- **PPO**: Stochastic policy → samples from $\mathcal{N}(\mu(s), \sigma(s))$
- **DDPG**: Deterministic policy → outputs $\mu(s)$ directly

### Exploration Strategy
- **PPO**: Intrinsic (policy entropy)
- **DDPG**: Extrinsic (OU noise added to actions)

### Stability
- **PPO**: More stable due to clipping mechanism
- **DDPG**: Can be unstable, needs careful tuning

### Key Innovations
**PPO:**
- Clipped surrogate objective prevents destructive policy updates
- Generalized Advantage Estimation (GAE) for variance reduction
- Multiple epochs on collected data

**DDPG:**
- Target networks for stability
- Replay buffer for sample efficiency
- Deterministic policy gradient

### Hyperparameters
**PPO Key Params:**
- `epsilon`: clip range (0.1-0.3)
- `gamma`: discount factor (0.9-0.99)
- `tau`: GAE parameter (0.8-0.95)
- `entropy_weight`: exploration bonus

**DDPG Key Params:**
- `gamma`: discount factor (0.95-0.99)
- `tau`: soft update rate (0.001-0.01)
- `ou_noise_theta`, `ou_noise_sigma`: exploration noise
- `initial_random_steps`: warm-up period

### When to Use?
**Use PPO when:**
- Stability is critical
- Sample efficiency is not a concern
- You want robust performance across diverse tasks

**Use DDPG when:**
- Sample efficiency matters
- Deterministic policies are preferred
- You have time to tune hyperparameters

### Expected Performance on Pendulum-v1
- **PPO**: Should converge to ~-200 to -150 reward
- **DDPG**: Should converge to ~-150 to -100 reward (often slightly better)

Both algorithms should learn to swing the pendulum up and balance it!

# Train & Test 

## Initialize

In [ ]:
# parameters
num_frames = 50000
memory_size = 20000
batch_size = 128
ou_noise_theta = 1.0
ou_noise_sigma = 0.1
initial_random_steps = 10000

ppo_agent = PPOAgent(
    env,
    gamma = 0.9,
    tau = 0.8,
    batch_size = 64,
    epsilon = 0.2,
    epoch = 64,
    rollout_len = 2048,
    entropy_weight = 0.005
)

ddpg_agent = DDPGAgent(
    env, 
    memory_size, 
    batch_size,
    ou_noise_theta,
    ou_noise_sigma,
    initial_random_steps=initial_random_steps
)

## Train PPO

In [ ]:
ppo_agent.train(num_frames)

## Train DDPG

In [ ]:
ddpg_agent.train(num_frames)

## Test

In [ ]:
# test
if IN_COLAB:
    ppo_agent.env = gym.wrappers.Monitor(ppo_agent.env, "videos", force=True)
    ddpg_agent.env = gym.wrappers.Monitor(ddpg_agent.env, "videos", force=True)
ppo_frames = ppo_agent.test()
ddpg_frames = ddpg_agent.test()

## Render

In [ ]:
if IN_COLAB:  # for colab
    import base64
    import glob
    import io
    import os

    from IPython.display import HTML, display

    def ipython_show_video(path: str) -> None:
        """Show a video at `path` within IPython Notebook."""
        if not os.path.isfile(path):
            raise NameError("Cannot access: {}".format(path))

        video = io.open(path, "r+b").read()
        encoded = base64.b64encode(video)

        display(HTML(
            data="""
            <video alt="test" controls>
            <source src="data:video/mp4;base64,{0}" type="video/mp4"/>
            </video>
            """.format(encoded.decode("ascii"))
        ))

    list_of_files = glob.glob("videos/*.mp4")
    latest_file = max(list_of_files, key=os.path.getctime)
    print(latest_file)
    ipython_show_video(latest_file)

else:  # for jupyter
    from matplotlib import animation
    from JSAnimation.IPython_display import display_animation
    from IPython.display import display


    def display_frames_as_gif(frames):
        """Displays a list of frames as a gif, with controls."""
        patch = plt.imshow(frames[0])
        plt.axis('off')

        def animate(i):
            patch.set_data(frames[i])

        anim = animation.FuncAnimation(
            plt.gcf(), animate, frames = len(frames), interval=50
        )
        display(display_animation(anim, default_mode='loop'))


    # display 
    display_frames_as_gif(ppo_frames)
    display_frames_as_gif(ddpg_frames)